In [1]:
# Descomentar para instalar última versión de PyTorch
# Se requiere verificar la compatibilidad en https://pytorch.org/get-started/locally/
#!pip install torch torchvision torchaudio

In [21]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [22]:
import torch
import torch.nn as nn

In [23]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

Escrito siguiendo las instrucciones en [Learning PyTorch with Examples](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#nn-module)

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [25]:
class TwoSigmoidModel(nn.Module):
    """
    Red neuronal con una capa oculta y sigmoides como funciones de activación.
    """
    
    def __init__(self, n_input = 2, n_hidden = 2, n_out = 1):
        """
        In the constructor we instantiate the nn.Linear modules and assign them as
        member variables.
        """
        super(TwoSigmoidModel, self).__init__()
        # El bias lo agrega PyTorch por defecto
        
        self.z1 = nn.Linear(n_input, n_hidden)
        self.z2 = nn.Linear(n_hidden, n_out)
        
    def forward(self, input):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors that can play the role of activation
        functions.
        
        Nota: La función _debe_ llamarse forward.
        """
        h1 = torch.sigmoid(self.z1(input))
        out = torch.sigmoid(self.z2(h1))
        return out

In [26]:
def show_w(axis, mat, name):
    cax = axis.matshow(mat.data.numpy())
    axis.set_title(name)
    return cax
    
def show_b(axis, mat, name):
    axis.matshow(mat.data.numpy().reshape((-1,1)))
    axis.set_title(name)

In [43]:
def train(model, X, Y, learning_rate, num_steps=500):
    """
    Recibe el modelo de red neuronal a entrenar,
    los datos de entrada X y los valores de salida deseados Y
    en tensores de PyTorch
    """
    errores = np.zeros(num_steps)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()              # Binary Cross Entropy
    #criterion = torch.nn.MSELoss()       # Mean Squared Error
    for t in range(num_steps):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(X)

        # Compute and print loss
        loss = criterion(y_pred, Y)
        #print(t, loss.item())
        errores[t] = loss.item()

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Grafica error
    p = plt.plot(np.arange(num_steps), errores)
    plt.title("Error")
    plt.ylabel("Entropía cruzada binaria")
    plt.xlabel("Iteración")
    
    # Grafica pesos
    parameters = list(model.parameters())
    fig, axes = plt.subplots(1,4, figsize=(12,1.5))
    w1 = parameters[0]
    b1 = parameters[1]
    w2 = parameters[2]
    b2 = parameters[3]
    if device != 'cpu':
        w1 = w1.cpu()
        b1 = b1.cpu()
        w2 = w2.cpu()
        b2 = b2.cpu()
    cax = show_w(axes[0], w1, "w1")
    show_b(axes[1], b1, "b1")
    show_w(axes[2], w2, "w2")
    show_b(axes[3], b2, "b2")
    fig.colorbar(cax)
    print("w1 = ", w1, " b1 = ", b1, " w2 = ", w2, " b2 = ", b2)

In [28]:
# TODO: Crear X, Y y ¡a usar!
X = torch.Tensor([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])
Y = torch.Tensor([[0],
                  [1],
                  [1],
                  [0]])

In [29]:
model = TwoSigmoidModel()

In [30]:
model.cuda()

TwoSigmoidModel(
  (z1): Linear(in_features=2, out_features=2, bias=True)
  (z2): Linear(in_features=2, out_features=1, bias=True)
)

In [32]:
next(model.z2.parameters()).device

device(type='cuda', index=0)

In [37]:
X = X.to(device)

In [38]:
Y = Y.to(device)

Antes del entrenamiento

In [39]:
Out = model.forward(X)

In [40]:
def print_state():
    print("X = ", X)
    print("Y = ", Y)
    print("Out = ", Out)
print_state()

X =  tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]], device='cuda:0')
Y =  tensor([[0.],
        [1.],
        [1.],
        [0.]], device='cuda:0')
Out =  tensor([[0.6447],
        [0.6427],
        [0.6214],
        [0.6196]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [44]:
@interact_manual(alpha=7)
def trainXOR(alpha):
    train(model, X, Y, learning_rate=alpha)
    global Out
    Out = model.forward(X)
    print_state()

interactive(children=(IntSlider(value=7, description='alpha', max=21, min=-7), Button(description='Run Interac…